# Training The NN Model

In [1]:
# import things
import sys
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

# change directory
os.chdir('/Users/syahrulhamdani/Desktop/thesis/predictive-maintenance-of-aircraft-engine/')
print('working directory in {}..'.format(os.getcwd()))

working directory in /Users/syahrulhamdani/Desktop/thesis/predictive-maintenance-of-aircraft-engine..


In [2]:
# import modules
from src.data import LoadData, list_dataset
from src.models import NN

In [3]:
datalist = list_dataset()
datalist

{'train_FD001': 'train_FD001.txt',
 'train_FD003': 'train_FD003.txt',
 'RUL_FD004': 'RUL_FD004.txt',
 'train_FD002': 'train_FD002.txt',
 'RUL_FD001': 'RUL_FD001.txt',
 'RUL_FD003': 'RUL_FD003.txt',
 'RUL_FD002': 'RUL_FD002.txt',
 'train_FD004': 'train_FD004.txt',
 'test_FD003': 'test_FD003.txt',
 'test_FD002': 'test_FD002.txt',
 'test_FD001': 'test_FD001.txt',
 'test_FD004': 'test_FD004.txt'}

In [4]:
with open('references/col_to_feat.json', 'r') as f:
    data = json.load(f)
feature_names = list(data.values())
feature_names

['EngineID',
 'Cycle',
 'OpSetting1',
 'OpSetting2',
 'OpSetting3',
 'T2',
 'T24',
 'T30',
 'T50',
 'P2',
 'P15',
 'P30',
 'Nf',
 'Nc',
 'epr',
 'Ps30',
 'phi',
 'NRf',
 'NRc',
 'BPR',
 'farB',
 'htBleed',
 'Nf_dmd',
 'PCNfR_dmd',
 'W31',
 'W32']

In [5]:
data = LoadData(datalist['train_FD001'], names=feature_names, sep='\s+')
print(data.features.shape, data.target.shape)

(20631, 26) (20631,)


In [6]:
# data.save_interim('data/interim', names=feature_names)

After doing preprocessing above, now we can define the training step as follows:
1. Define the model with required arguments
2. Initialize model hyperparameters
3. Initialize criterion and optimizer
4. Build the pipeline

In [7]:
data.features[:, 2:].shape

(20631, 24)

In [8]:
# Convert to tensor (both features and targets)
featureset = torch.from_numpy(data.features[:,2:])
labelset = torch.from_numpy(data.target)

# Load into dataloader (create loader both for feature and label without shuffling when loading)
featureloader = torch.utils.data.DataLoader(featureset, batch_size=26)
labelloader = torch.utils.data.DataLoader(labelset, batch_size=26)

In [38]:
def training(
    model,
    criterion,
    optimizer,
    featureloader,
    labelloader,
    epochs=5,
    print_every=40
):
    epoch_loss = 0
    steps = 0
    
    for epoch in range(epochs):
        for features, labels in zip(featureloader, labelloader):
            steps += 1
            features, labels = features.type(torch.FloatTensor), labels.type(torch.FloatTensor)
            labels.resize_(labels.shape[0], 1)
            
            output = model.forward(features)
            loss = criterion(output, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            
            if steps % print_every == 0:
                print('Epoch: {}/{}..'.format(epoch+1, epochs),
                      'Training loss: {:.3f}'.format(epoch_loss/print_every))

                epoch_loss = 0

In [43]:
class NN(nn.Module):
    """Neural Network Generator."""
    def __init__(self, input_size, output_size, drop_p, hidden_sizes=[30, 15]):
        """Generate fully-connected neural network.

        parameters
        ----------
        input_size (int): size of the input
        hidden_sizes (list of int): size of the hidden layers
        output_layer (int): size of the output layer
        drop_p (float): dropout probability
        """
        super().__init__()
        self.hidden_layers = nn.ModuleList([
            nn.Linear(input_size, hidden_sizes[0])
        ])
        layers = zip(hidden_sizes[:-1], hidden_sizes[1:])
        self.hidden_layers.extend([nn.Linear(h1,h2) for h1,h2 in layers])
        self.output = nn.Linear(hidden_sizes[-1], output_size)
        self.dropout = nn.Dropout(drop_p)
    
    def forward(self, X):
        X = F.sigmoid(self.hidden_layers[0](X))
        X = F.relu(self.hidden_layers[1](X))
        X = self.output(X)

        return F.relu(X)

In [49]:
model = NN(data.features[:,2:].shape[1], 1, 0.6)
model

NN(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=24, out_features=30, bias=True)
    (1): Linear(in_features=30, out_features=15, bias=True)
  )
  (output): Linear(in_features=15, out_features=1, bias=True)
  (dropout): Dropout(p=0.6)
)

In [50]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.1207,  0.0471,  0.1043,  0.0299, -0.0484, -0.1376, -0.0278,  0.1144,
          0.1431, -0.1252, -0.0893,  0.1281, -0.0762, -0.0864,  0.1334,  0.0138,
          0.1178, -0.1651, -0.1836, -0.1306, -0.0981,  0.1644,  0.0123,  0.1227],
        [ 0.0880, -0.1577, -0.1467,  0.1576, -0.0168, -0.1775, -0.1326,  0.1045,
          0.0675,  0.1972,  0.1339, -0.0182,  0.0996, -0.1759,  0.1979,  0.0365,
         -0.1898,  0.1890,  0.0852, -0.0679,  0.0930, -0.1474, -0.1149,  0.1184],
        [-0.1668, -0.2031,  0.1146, -0.0452,  0.0125,  0.0664,  0.1021, -0.2012,
          0.0670,  0.1502,  0.0364,  0.1192,  0.1024, -0.0330, -0.0641, -0.0269,
          0.2003,  0.0928,  0.0284, -0.1100, -0.1588,  0.2030, -0.0438,  0.0484],
        [ 0.0156, -0.0329,  0.1410,  0.0768, -0.1651, -0.0024,  0.1840,  0.1567,
          0.0768,  0.1071,  0.1930,  0.1541, -0.1584,  0.0420,  0.1902, -0.0860,
         -0.0718, -0.1903,  0.0810,  0.1797,  0.0189,  0.1516, -0.1392,  0.1323],
  

In [51]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

training(model, criterion, optimizer, featureloader, labelloader, epochs=50)

/Users/syahrulhamdani/anaconda3/envs/100DaysOfMLCode/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 1/50.. Training loss: 19789.023
Epoch: 1/50.. Training loss: 14793.422
Epoch: 1/50.. Training loss: 12829.431
Epoch: 1/50.. Training loss: 16141.982
Epoch: 1/50.. Training loss: 13074.153
Epoch: 1/50.. Training loss: 10904.154
Epoch: 1/50.. Training loss: 10493.633
Epoch: 1/50.. Training loss: 11931.201
Epoch: 1/50.. Training loss: 14308.731
Epoch: 1/50.. Training loss: 12547.993
Epoch: 1/50.. Training loss: 14376.928
Epoch: 1/50.. Training loss: 9279.416
Epoch: 1/50.. Training loss: 19270.188
Epoch: 1/50.. Training loss: 18794.553
Epoch: 1/50.. Training loss: 11073.011
Epoch: 1/50.. Training loss: 12195.018
Epoch: 1/50.. Training loss: 15375.745
Epoch: 1/50.. Training loss: 14816.861
Epoch: 1/50.. Training loss: 18590.410
Epoch: 2/50.. Training loss: 6220.941
Epoch: 2/50.. Training loss: 12141.911
Epoch: 2/50.. Training loss: 9545.015
Epoch: 2/50.. Training loss: 5415.592
Epoch: 2/50.. Training loss: 8955.735
Epoch: 2/50.. Training loss: 5706.934
Epoch: 2/50.. Training loss: 51

Epoch: 12/50.. Training loss: 4757.614
Epoch: 12/50.. Training loss: 3736.756
Epoch: 12/50.. Training loss: 4438.272
Epoch: 12/50.. Training loss: 3359.463
Epoch: 12/50.. Training loss: 3533.952
Epoch: 12/50.. Training loss: 3441.559
Epoch: 12/50.. Training loss: 3271.716
Epoch: 12/50.. Training loss: 3847.918
Epoch: 12/50.. Training loss: 3957.938
Epoch: 12/50.. Training loss: 5262.359
Epoch: 12/50.. Training loss: 3225.813
Epoch: 12/50.. Training loss: 5617.149
Epoch: 12/50.. Training loss: 9057.895
Epoch: 12/50.. Training loss: 3741.464
Epoch: 12/50.. Training loss: 3778.801
Epoch: 12/50.. Training loss: 5682.094
Epoch: 12/50.. Training loss: 4528.731
Epoch: 12/50.. Training loss: 8641.961
Epoch: 12/50.. Training loss: 6345.141
Epoch: 13/50.. Training loss: 4538.139
Epoch: 13/50.. Training loss: 4722.704
Epoch: 13/50.. Training loss: 4015.152
Epoch: 13/50.. Training loss: 4506.376
Epoch: 13/50.. Training loss: 3590.794
Epoch: 13/50.. Training loss: 3514.967
Epoch: 13/50.. Training l

Epoch: 22/50.. Training loss: 5437.632
Epoch: 22/50.. Training loss: 6644.490
Epoch: 22/50.. Training loss: 8289.767
Epoch: 23/50.. Training loss: 4666.959
Epoch: 23/50.. Training loss: 4633.116
Epoch: 23/50.. Training loss: 3933.629
Epoch: 23/50.. Training loss: 4382.436
Epoch: 23/50.. Training loss: 3445.454
Epoch: 23/50.. Training loss: 3474.760
Epoch: 23/50.. Training loss: 3393.302
Epoch: 23/50.. Training loss: 3089.553
Epoch: 23/50.. Training loss: 3543.303
Epoch: 23/50.. Training loss: 4252.481
Epoch: 23/50.. Training loss: 4188.574
Epoch: 23/50.. Training loss: 4541.446
Epoch: 23/50.. Training loss: 4369.019
Epoch: 23/50.. Training loss: 10206.293
Epoch: 23/50.. Training loss: 3239.933
Epoch: 23/50.. Training loss: 3852.158
Epoch: 23/50.. Training loss: 6268.648
Epoch: 23/50.. Training loss: 4331.421
Epoch: 23/50.. Training loss: 7121.279
Epoch: 23/50.. Training loss: 7819.306
Epoch: 24/50.. Training loss: 4703.499
Epoch: 24/50.. Training loss: 4634.897
Epoch: 24/50.. Training 

Epoch: 33/50.. Training loss: 4816.107
Epoch: 33/50.. Training loss: 3267.227
Epoch: 33/50.. Training loss: 6845.013
Epoch: 33/50.. Training loss: 8025.360
Epoch: 33/50.. Training loss: 3629.899
Epoch: 33/50.. Training loss: 4083.004
Epoch: 33/50.. Training loss: 6701.633
Epoch: 33/50.. Training loss: 3370.225
Epoch: 33/50.. Training loss: 8499.380
Epoch: 33/50.. Training loss: 6196.996
Epoch: 34/50.. Training loss: 5652.406
Epoch: 34/50.. Training loss: 4072.199
Epoch: 34/50.. Training loss: 3548.360
Epoch: 34/50.. Training loss: 4622.970
Epoch: 34/50.. Training loss: 3772.617
Epoch: 34/50.. Training loss: 3216.035
Epoch: 34/50.. Training loss: 3162.764
Epoch: 34/50.. Training loss: 3580.761
Epoch: 34/50.. Training loss: 4211.072
Epoch: 34/50.. Training loss: 3622.419
Epoch: 34/50.. Training loss: 4887.367
Epoch: 34/50.. Training loss: 3127.225
Epoch: 34/50.. Training loss: 7013.123
Epoch: 34/50.. Training loss: 7772.431
Epoch: 34/50.. Training loss: 4001.648
Epoch: 34/50.. Training l

Epoch: 44/50.. Training loss: 4020.906
Epoch: 44/50.. Training loss: 4408.030
Epoch: 44/50.. Training loss: 3439.790
Epoch: 44/50.. Training loss: 3450.278
Epoch: 44/50.. Training loss: 3512.019
Epoch: 44/50.. Training loss: 3007.709
Epoch: 44/50.. Training loss: 3450.913
Epoch: 44/50.. Training loss: 4387.253
Epoch: 44/50.. Training loss: 4231.926
Epoch: 44/50.. Training loss: 4333.720
Epoch: 44/50.. Training loss: 4571.893
Epoch: 44/50.. Training loss: 10025.141
Epoch: 44/50.. Training loss: 3771.309
Epoch: 44/50.. Training loss: 3393.687
Epoch: 44/50.. Training loss: 6132.293
Epoch: 44/50.. Training loss: 4577.102
Epoch: 44/50.. Training loss: 7217.122
Epoch: 44/50.. Training loss: 7614.037
Epoch: 45/50.. Training loss: 4597.845
Epoch: 45/50.. Training loss: 4747.915
Epoch: 45/50.. Training loss: 3960.356
Epoch: 45/50.. Training loss: 4295.445
Epoch: 45/50.. Training loss: 3574.156
Epoch: 45/50.. Training loss: 3569.625
Epoch: 45/50.. Training loss: 3478.731
Epoch: 45/50.. Training 